<a href="https://colab.research.google.com/github/BrunaCostaDTS/BrunaCostaDTS/blob/main/Hotmart_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Library**

In [ ]:
!pip install pandas

In [ ]:
!pip install pandasql
!pip install pandas

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=93abf11666e97d18732618207a874151aea6ceb105013daa2408127e71b5ec5c
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


**Imports**

In [ ]:
import pandas as pd
import pandasql as psql

**Loads**

In [15]:
# Carregar CSV
df_Producers = pd.read_csv('/content/Hotmart/Producers.csv')
df_Products = pd.read_csv('/content/Hotmart/Products.csv')
df_Sales = pd.read_csv('/content/Hotmart/Sales.csv')
#df_Purchases = pd.read_csv('Purchases.csv')

**Queries**



---



1. **O produto top com mais compras válidas (sem reembolso, cancelamento ou estorno), em cada nicho com área
de adesão desativada e recuperação ativada.**


1. *PS: Entendo nessa questão que devo trazer o produto mais vendido de cada nicho (sem reembolso, cancelamento ou estorno), desde que a area de adesão (member_area_active) esteja desativada e a área de recuperação(recovery_active) esteja ativada.*

In [30]:
# Fazer uma consulta SQL
query = """
with cte as(
SELECT
    a.product_id
    ,b.niche
    ,count(*) as TotalCompras
FROM df_Sales a
left JOIN df_Products b on b.product_id = a.product_id
WHERE
    a.refund = 0
    and a.cancelled = 0
    and a.chargeback = 0
    and b.member_area_active = 0
    and b.recovery_active = 1
GROUP by  a.product_id, b.niche
)
,Rank as (
SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY niche ORDER BY TotalCompras DESC) AS rank
FROM cte
)
SELECT
    a.niche
    ,a.product_id
    ,a.TotalCompras
from Rank a
where rank = 1
"""
result = psql.sqldf(query, globals())
print(result)

                   niche  product_id  TotalCompras
0       Empreendedorismo   983844542           333
1               Financas   348532296            49
2    Saúde e Alimentação   260868848           148
3  Tecnologia e Inovação   719732475           169
4                Viagens   324917616            92




---



**2. Os 5 principais produtores que aderiram à Hotmart a partir de 2019 e obtiveram a maior comissão
usando a recuperação.**

*2. PS: Entendo que devo retornar o top 5 produtores que entraram como clientes da hotmart a partir de 2019 e obtiveram a maior comissão(comission_value) usando a recuperação (recovery_active).*

In [31]:
# Fazer uma consulta SQL
# Esse exemplo usando a mesma query e raciocínio usados no dash e no data.world não reconheceu o produtor 13 como o primeiro da lista, logo, ele puxou o seguinte (10)
query = """
SELECT
    c.producer_id
    ,SUM(a.comission_value)
FROM df_Sales a
LEFT JOIN df_Products b on a.product_id = b.product_id
LEFT JOIN df_Producers c on b.producer_id = c.producer_id
WHERE
    b.recovery_active = 1
    and c.registry_date >= '2019-01-01'
GROUP BY c.producer_id
ORDER BY SUM(a.comission_value) desc
LIMIT 5
"""
result = psql.sqldf(query, globals())
print(result)

   producer_id  SUM(a.comission_value)
0            3                293341.1
1            6                250124.6
2            7                128060.4
3            5                 45328.3
4           10                 35700.0




---



**3. Liste o número de vendas realizadas em cada dia da semana dos anos de 2020, 2021 e
2022 e a variação entre as vendas dos dias da semana de 2021/2020 e 2022/2021.**

*3. Entendi que preciso retornar o numero de vendas realizadasem cada semana de 2020 a 2022, junto com a variação entre as vendas dos dias da semana de 2021/2020 e 2022/2021.*

In [ ]:
df_Producers = pd.read_csv('/content/Hotmart/Producers.csv')
df_Products = pd.read_csv('/content/Hotmart/Products.csv')
df_Sales = pd.read_csv('/content/Hotmart/Sales.csv')